In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

import cv2
import os
import glob
# use gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

import math
import scipy as sp
import PIL
from PIL import Image, ImageOps
import copy
import itertools

# Tensorflow
import tensorflow as tf
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# split dataset
import splitfolders

# ignore warnings
import warnings
warnings.filterwarnings(action='ignore')

# flask
from flask import Flask
from flask import render_template
from flask import request

# json 
import json
import xmltodict
from json2xml import json2xml
from pandas.io.json import json_normalize
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, ElementTree

In [2]:
def json_to_dataframe(folder_dir) :
    file_list = os.listdir(folder_dir)
    df = pd.DataFrame()
    for i in file_list:
        with open(folder_dir+i, 'r') as f:
            dict = json.loads(f.read())
            df1 = pd.json_normalize(dict['description']) 
            df2 = pd.json_normalize(dict['annotations']) 
            df3 = pd.DataFrame(pd.concat([df1,df2], axis=1))
            df = pd.concat([df,df3], axis=0)
        df.reset_index(drop=True, inplace=True)
    return df

In [3]:
def json2xml(folder_dir, destination):
    file_list = os.listdir(folder_dir)
    for i in file_list:
        with open(folder_dir+i, 'r') as f:
            data = json.loads(f.read()) 
        root = ET.Element("root")
        desc = ET.SubElement(root, "description")
        ET.SubElement(desc, "image").text = str(data["description"]["image"])
        ET.SubElement(desc, "height").text = str(data["description"]["height"])
        ET.SubElement(desc, "width").text = str(data["description"]["width"])
        
        annot = ET.SubElement(root, "annotations")    
        ET.SubElement(annot, "disease").text = str(data["annotations"]["disease"])
        ET.SubElement(annot, "crop").text = str(data["annotations"]["crop"])
        ET.SubElement(annot, "area").text = str(data["annotations"]["area"])
        ET.SubElement(annot, "grow").text = str(data["annotations"]["grow"])
        ET.SubElement(annot, "risk").text = str(data["annotations"]["risk"])
        ET.SubElement(annot, "points").text = str(data["annotations"]["points"][0])
        tree = ET.ElementTree(root)
        tree.write(destination + i + ".xml")
    print("Conversion completed")
        
    file_names = os.listdir(destination)
    for name in file_names:
        src = os.path.join(destination, name)
        dst = os.path.basename(name).replace('.jpg.json.xml', '.xml').replace('.JPG.json.xml', '.xml')
        dst = os.path.join(destination, dst)
        os.rename(src, dst)
    print("Renaming completed")

In [264]:
json2xml('./Training/새 폴더/', './Training/새 폴더 (2)/')

TypeError: Element() argument 2 must be dict, not str

In [4]:
json2xml('./Training/원본/[라벨]사과_1.질병/', './static/xml/Train/사과_1.질병/')


KeyboardInterrupt



In [262]:
json2xml('./Validation/원본/[라벨]사과_1.질병/', './static/xml/Test/사과_1.질병/')

Conversion completed
Renaming completed


In [251]:
xmls_root ="./static/xml/Train/배_0.정상/"  
jpegs = './Training/원본/배_0.정상/'
XMLs = os.listdir(xmls_root)
ix = np.random.randint(len(XMLs))
sample_xml = XMLs[ix]
sample_xml = '{}/{}'.format(xmls_root, sample_xml)
with open(sample_xml, "rb") as f:    
    d = xmltodict.parse(f, xml_attribs=True)

d

{'root': {'description': {'image': 'V006_80_0_00_01_05_26_0_c16_20201207_0029_S01_1.JPG'},
  'annotations': {'disease': '0',
   'crop': '1',
   'area': '5',
   'grow': '26',
   'risk': '0',
   'points': "{'xtl': 1067, 'ytl': 0, 'xbr': 2112, 'ybr': 4031}"}}}

In [33]:
# Labeled data folder paths 
pear0_path = './Training/원본/[라벨]배_0.정상/'
pear1_path = './Training/원본/[라벨]배_1.질병/'
apple0_path = './Training/원본/[라벨]사과_0.정상/'
apple1_path = './Training/원본/[라벨]사과_1.질병/'

In [34]:
pear0_df = json_to_dataframe(pear0_path)
pear1_df = json_to_dataframe(pear1_path)
apple0_df = json_to_dataframe(apple0_path)
apple1_df = json_to_dataframe(apple1_path)